## 加载环境变量

In [1]:
import os
from dotenv import load_dotenv

# 加载 .env 文件中的环境变量
load_dotenv(override=True)  # 使用 override=True 确保加载最新的 .env 数据

True

## 定义 LLM (大语言模型)
- 实例化一个 ChatOpenAI 对象，它将作为我们应用的核心。我们从环境变量中获取 API 密钥和模型名称，以保证安全性。

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model=os.environ.get("OPENAPI_MODEL"),
    openai_api_base=os.environ.get("OPENAPI_API_BASE"),
    openai_api_key=os.environ.get("OPENAPI_API_KEY"),
    temperature=0.0,
)

## 自定义提示词模板
+ 实现提示词的动态化：将提示词中的内容抽象为变量，这样你就可以根据不同的输入，动态地生成新的提示词，而无需硬编码。这使得你的代码更加灵活和可复用。
+ 确保提示词的一致性：无论你如何改变输入，提示词的结构和意图都是固定的。这能有效避免因手动修改提示词而引入的错误，确保大模型始终接收到你期望的指令。
+ 提高可维护性：当你需要修改提示词的结构或指令时，只需修改一处模板即可，所有使用该模板的地方都会自动更新。

In [3]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "你是一个起名大师，请模仿示例起3个{county}特色的名字，比如男孩经常被叫做{boy}，女孩被经常叫做{girl}。请返回以逗号分割的列表形式，不要返回其他内容"
)

### 提示词模板的使用示例

In [36]:
formatted_prompt = prompt_template.format(county="中国", boy="狗蛋", girl="翠花")
print(f"Formatted Prompt:\n{formatted_prompt}")

Formatted Prompt:
你是一个起名大师，请模仿示例起3个中国特色的名字，比如男孩经常被叫做狗蛋，女孩被经常叫做翠花。请返回以逗号分割的列表形式，不要返回其他内容


## 自定义输出解释器
+ 将非结构化文本转化为结构化数据：大语言模型（LLM）的输出通常是纯文本，而应用程序往往需要结构化数据，如 JSON、列表或字典。输出解析器能够将 LLM 生成的文本自动解析成我们需要的格式。
+ 提高大模型的可靠性：通过在提示词中注入格式化指令，解析器能更精确地引导大模型生成符合要求的输出，大大减少了模型的“幻觉”和格式错误。
+ 简化后续处理：一旦输出被解析成 Python 对象，你就可以像处理任何普通数据一样对它进行操作，无需进行复杂的字符串处理，这使得下游代码更加简洁。

In [5]:
from langchain_core.output_parsers import BaseOutputParser


class CommaSpearatedListOutputParser(BaseOutputParser):
    """将LLM生成的文本转化为序列数据"""

    def parse(self, text: str):
        """将LLM生成的文本转化为序列数据"""
        return text.strip().split(", ")

### 输出解析器的使用示例


In [38]:
result = CommaSpearatedListOutputParser().parse(text="hello, world, foo, bar")
print(f"Parsed Result:\n{result}")

Parsed Result:
['hello', 'world', 'foo', 'bar']


## 模型、提示词模板、输出解析器组合使用实例

### 手动调用
- 展示了 LangChain 的底层工作原理。我们手动完成提示词的格式化、调用 LLM 和解析返回结果。

In [7]:
formatted_prompt = prompt_template.format(county="中国", boy="狗蛋", girl="翠花")
print(f"Formatted Prompt:\n{formatted_prompt}")

llm_response = llm.invoke(formatted_prompt)
print(f"LLM Response:\n{llm_response}")

parsed_response = CommaSpearatedListOutputParser().parse(text=llm_response.content)
print(f"Parsed Response:\n{parsed_response}")

Formatted Prompt:
你是一个起名大师，请模仿示例起3个中国特色的名字，比如男孩经常被叫做狗蛋，女孩被经常叫做翠花。请返回以逗号分割的列表形式，不要返回其他内容
LLM Response:
content='铁柱,二丫,栓子' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 48, 'total_tokens': 56, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 0, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'deepseek-v3-250324', 'system_fingerprint': None, 'id': '02175645665939795a4d016b90785da794806a35be385267a3fbd', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--6c2ebc82-0ccb-4816-8e00-c25e8a410127-0' usage_metadata={'input_tokens': 48, 'output_tokens': 8, 'total_tokens': 56, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}}
Parsed Response:
['铁柱,二丫,栓子']


### 使用 LLMChain
- 我们将所有组件（模型、提示词模板、输出解析器）串联起来，形成一个完整的 Chain
- LLMChain 将上述所有手动步骤封装成一个可复用的流程。你只需要传入输入变量，Chain 就会自动完成所有工作。

In [48]:
from langchain.chains import LLMChain

llm_chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
    output_parser=CommaSpearatedListOutputParser(),
)

chain_input = {"county": "中国", "boy": "狗蛋", "girl": "翠花"}

response_from_chain = llm_chain.invoke(chain_input)

print(f"Response:\n{response_from_chain}")

Response:
{'county': '中国', 'boy': '狗蛋', 'girl': '翠花', 'text': ['铁柱,二丫,大牛']}
